In [2]:
import gridmet_cfsv2 as gm
import grd2shp_xagg
import geopandas as gpd
import numpy as np


gm_vars = ['air_temperature',
           'air_temperature',
           'precipitation_amount',
           'specific_humidity']
m = gm.Gridmet(type=3)

In [3]:
ds = m.tmax